#### Author: Yuan(Brian) Hu

# Text classification using Multinomial Naive Bayes on 20newsgroup

   In this notebook, I will go through text classification task by implementing ``Multinomial Naive Bayes Classifier`` from scratch, the data can be downloaded from [20news-bydate-matlab.tgz](http://qwone.com/~jason/20Newsgroups/20news-bydate-matlab.tgz) and [vocabulary.txt](http://qwone.com/~jason/20Newsgroups/vocabulary.txt). 

#  Objectives and deliverables:
- How does Multinomial Naive Bayes Classifier work?
- Class implementation of ``Multinomial Naive Bayes Classifier``
- Improve model performance(Accuracy) by replacing ``term-frequency`` f with log(1+f) and introducing ``TF-IDF``

<br><br/>

## <div align="center"> Multinomial Navie Bayes in text classfication</div>

#### ``Multinomial`` distribution over a vocabulary $V$:

<div align="center">$p = (p_1,...,p_{|V|})$, such that $p_i\geq  0$ and $\sum_i P_i =1$</div>

#### Document $x = (w_1,...w_{|V|})$ has probability     $\propto p_1^{w_1}p_2^{w_2} \cdot \cdot \cdot p_{|V|}^{w_{|V|}}$

#### Fitting navie Bayes: one multinomial distribution per class.
- ``Prior probabilities``: Class probabilities $\pi_1, ... , \pi_k$
- ``Likelihood``: Multinomials $p^1 = (p_{11},...,p_{1|V|})$, ... , $p^k = (p_{k1},...,p_{j|V|})$

#### Classifying document x by ``MAP estimator``:
<div align="center">$\underset{j}{\operatorname{argmax}} \pi_j \Pi^{|V|}_{i=1} P_{ji}^{X_i}$</div>
equals to:

<div align="center">$\underset{j}{\operatorname{argmax}} log(\pi_j)+\sum^{|V|}_{i}X_i \cdot log P_j(X_i)$</div>

The words in the documents constitute an overall vocabulary V of size 61188. All the documents will have to be classified into 20 classes. For each of the 20 classes $j = 1, 2, . . . , 20$, we are going to calculate the following:

- $\pi_j$ , the fraction of documents that belong to that class; and
- $P_j(X)$ , a probability distribution over V that models the documents of that class.

In order to fit $P_j(X)$ , imagine that all the documents of class $j$ are strung together. For each word
$w \in V$ in the document $X_i$ , let $P_j(X_i)$ be the fraction of this concatenated document occupied by $w$. We
will need to do smoothing (just add one to the count of how often $w$ occurs).

<br><br/>

## Data Flows:


 1. Load training/testing data into DataFrame
 

 2. Transform training/testing data to get ``BOW(Bag-of-Words)``matrix
 

 3. Estimate ``prior probability`` $\pi_j$ and ``likelihood probability`` $P_j(X_i)$ (multinomials)
 
 
 4. Classify documents $X_i$ by estimating $\underset{j}{\operatorname{argmax}} log(\pi_j)+\sum^{|V|}_{i}X_i* log P_j(X_i)$


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Step 1: Load data into DataFrame

In [2]:
def data_processing(data_path, label_path):
    #process labels
    labels = pd.read_csv(label_path, header = None)
    labels = labels.rename(columns={0: "Class"})
    labels.index = labels.index + 1
    labels.index.names = ['docIdx']
    
    #process data
    data = pd.read_csv(data_path, delimiter = " ", header = None)
    data.rename(columns={0: "docIdx", 1: "wordIdx", 2: "Count"}, inplace=True) 
    
    
    return data, labels

def count_V(filepath):
    f = open(filepath, 'r')
    words = f.readlines()
    return len(words)

#### Load training data

In [3]:
#load and preprocess training data
train_data = 'data/20news-bydate 2/matlab/train.data'
train_label = 'data/20news-bydate 2/matlab/train.label'

data, labels = data_processing(train_data, train_label)

In [4]:
#merge data with labels
data = data.merge(labels, how='left', on='docIdx')

In [5]:
data.head(30).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
docIdx,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
wordIdx,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Count,4,2,10,4,2,1,1,1,3,9,...,1,1,54,3,3,1,11,2,90,20
Class,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
#check if docIdx gets correct Class label
data.groupby("docIdx").Class.head()

0           1
1           1
2           1
3           1
4           1
           ..
1467133    20
1467134    20
1467135    20
1467136    20
1467137    20
Name: Class, Length: 56315, dtype: int64

#### Load testing data

In [7]:
#load and preprocess testing data
test_data = 'data/20news-bydate 2/matlab/test.data'
test_labels = 'data/20news-bydate 2/matlab/test.label'

In [8]:
test, test_labels = data_processing(test_data, test_labels)

<br/><br/>

### Step 2: Transform DataFrames to get ``Bag-of-Words``

``The reason Naive Bayes is naive``, is that it does not take word order or phrasing into account.

"In other words, Naive Bayes would give the exact same probability to the phrase "I like 
pizza" as it would to the phrase "Pizza like I". Even though people frequently say "I like pizza" and almost never say "Pizza like I". Because keeping track of every phrase and word ordering would be impossible, Naive Bayes doesn’t even try. That said, Naive Bayes works well in practice, so keeping track of word order must not be super important."   -StatsQuest

In regarding to likelihood of each word, it makes ``naive`` assumption that each word are uncorrelated.

We need to build a ``Bag of Words`` matrix from our DataFrame for the following modeling steps, first, we need acquire the length of Vocabulary and the number of documents respectively.

We will take the outputs of this step as our ``X_train`` and ``X_test`` respectively.

In [9]:
#number of documents - as number of samples(index starts from 1 to 11269)
n_docs = data.docIdx.nunique()
n_docs

11269

In [10]:
#number of words
V_path = 'data/vocabulary.txt'

n_words = count_V(V_path)
print(f"The total number of words in vocabulary.txt is {n_words}")

The total number of words in vocabulary.txt is 61188


In [11]:
def bag_of_words(df, n_words):
    """
    dataframe: DataFrame("docIdx","wordIdx","Count","Class")
    n_words: the number of unique words in "vocabulary.txt"
    
    word_mat: word count matrix of shape (n_documents, n_words)
              this result also will serve us as X_train
    """
    n_docs = df.docIdx.nunique()

    word_mat = np.zeros((n_docs, n_words))
    for i in range(n_docs):
        doc = df[df.docIdx==i+1]
        word_vec = np.zeros((n_words, ))
        mask = doc.wordIdx.values-1
        word_vec[mask] = doc.Count.values
        word_mat[i]=word_vec
        
    return word_mat

#### Transform training DataFrame

In [13]:
#initialize X_train
X_train = bag_of_words(data, n_words)
X_train

array([[ 4.,  2., 10., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [14]:
X_train.shape

(11269, 61188)

In [15]:
#checking the matrix correctly reflects dataframe's values
print(np.unique(X_train[0]))
print(np.unique(data[data.docIdx==1].Count))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 13. 15. 19. 20. 24. 27.
 54. 58. 90.]
[ 1  2  3  4  5  6  7  8  9 10 11 13 15 19 20 24 27 54 58 90]


Since we will list all the words with zero count in the matrix, the results above matches our expectation.

In [16]:
#initialize y_train
y_train = labels.Class.values
y_train

array([ 1,  1,  1, ..., 20, 20, 20])

In [17]:
#number of classes
n_classes = np.unique(y_train).shape[0]
n_classes

20

#### Transform testing DataFrame

In [18]:
#initialize X_test
X_test = bag_of_words(test, n_words)
X_test.shape

(7505, 61188)

In [19]:
#checking the matrix correctly reflects dataframe's values
print(np.unique(X_test[0]))
print(np.unique(test[test.docIdx==1].Count))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 12.]
[ 1  2  3  4  5  6  7  8  9 10 12]


In [21]:
#initialize y_test
y_test = test_labels.Class.values
y_test

array([ 1,  1,  1, ..., 20, 20, 20])

<br/><br/>

### Step 3: Estimate ``prior probability`` $\pi_j$ and ``likelihood probability`` $P_j(X_i)$ (multinomials)

#### Estimate ``prior probability`` $\pi_j$

> Sum of all the prior probabilities should equals to 1

In [22]:
#number of documents in each class
n_docs_class = np.array([y_train[y_train==i].shape[0] for i in range(1, 21)])
n_docs_class

array([480, 581, 572, 587, 575, 592, 582, 592, 596, 594, 598, 594, 591,
       594, 593, 599, 545, 564, 464, 376])

In [23]:
#the fraction of documents that belong to that class
prior = n_docs_class/np.sum(n_docs_class)

In [24]:
prior

array([0.04259473, 0.05155737, 0.05075872, 0.0520898 , 0.05102494,
       0.0525335 , 0.05164611, 0.0525335 , 0.05288846, 0.05271098,
       0.05306593, 0.05271098, 0.05244476, 0.05271098, 0.05262224,
       0.05315467, 0.04836277, 0.05004881, 0.0411749 , 0.03336587])

In [25]:
#check constraint
np.sum(prior)

1.0

In [26]:
#alternative with np.unique()
n,y_counts = np.unique(y_train, return_counts=True)
#print(n,y_counts)
phi_y =  y_counts/y_counts.sum()
phi_y

array([0.04259473, 0.05155737, 0.05075872, 0.0520898 , 0.05102494,
       0.0525335 , 0.05164611, 0.0525335 , 0.05288846, 0.05271098,
       0.05306593, 0.05271098, 0.05244476, 0.05271098, 0.05262224,
       0.05315467, 0.04836277, 0.05004881, 0.0411749 , 0.03336587])

#### Estimate ``likelihood probability`` $P_j(X_i)$

> Constraint: Sum of likelihood probabilities of each class should equals to 1 as well

> ``Smoothing``: we will use add-one smoothing, which sets alpha to be 1

$$P(X_j|w_i)=\frac{\sum^{|V|}_{i}tf(X_j, w_i \in d_j) + \alpha}{\sum^{|V|}_{i}N+ \alpha*|V|}$$

In [27]:
#calculate multinomial probability of each word within each document class
#returns a matrix which has the same shape with tf matrix
def multinomials(X_train, y_train):
    tf = np.zeros((n_classes, n_words))
    #X_train = X_train + 1
    for j in range(20):
        jth_class = X_train[y_train==j+1]
        f = (np.sum(jth_class, axis=0)+1)/(np.sum(jth_class)+n_words)
        tf[j] = f
    return tf

In [28]:
likelihood = multinomials(X_train, y_train)

In [29]:
likelihood

array([[6.66666667e-05, 3.04761905e-04, 1.31428571e-03, ...,
        4.76190476e-06, 4.76190476e-06, 4.76190476e-06],
       [3.55589754e-04, 3.49760414e-04, 5.82934024e-06, ...,
        5.82934024e-06, 5.82934024e-06, 5.82934024e-06],
       [7.89707479e-05, 4.60662696e-04, 6.58089566e-06, ...,
        6.58089566e-06, 6.58089566e-06, 6.58089566e-06],
       ...,
       [3.48108977e-05, 4.90517195e-04, 3.16462706e-06, ...,
        3.16462706e-06, 3.16462706e-06, 3.16462706e-06],
       [4.03854386e-06, 1.61541755e-04, 4.03854386e-06, ...,
        4.03854386e-06, 4.03854386e-06, 4.03854386e-06],
       [5.54680393e-06, 2.55152981e-04, 5.54680393e-05, ...,
        5.54680393e-06, 5.54680393e-06, 5.54680393e-06]])

In [30]:
#check constraint 
np.sum(likelihood, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

### Step 4: Estimate 
<div align="center">$\underset{j}{\operatorname{argmax}} log(\pi_j)+\sum^{|V|}_{i}X_i* log P_j(X_i)$</div>

In [207]:
#example of classify a single sample X_test[0]
probs = []
for j in range(20):
    prob = np.sum(X_test[2]*np.log(likelihood[j]))+np.log(prior[j])
    probs.append(prob)
    
np.argmax(probs)+1

1

For test sample X_test[0], the predicted result is class"1".

Based on the logic above we write our estimator function, we will take advantage of numpy broadcasting for 2D-matrix computation.

In [31]:
# calculate optimal solution for estimator
def estimator(X_test, prior, likelihood):
    if X_test.ndim == 1:
        n_documents = 1
    else:
        n_documents = X_test.shape[0]

    labels = np.zeros((n_documents, ))
    for i in range(n_documents):
        log_prob = np.sum(np.log(likelihood)*X_test[i], axis=1) + np.log(prior)
        labels[i] = np.argmax(log_prob)+1
    return labels

<br/><br/>

# Model Improving

Before we wrap up the class implementation, we will take these two strategies to try to imporve our model.

In [ ]:
#splitting training data into training and validation set
from sklearn.model_selection import train_test_split

X_train_, X_valid, y_train_, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [35]:
X_train_.shape

(9015, 61188)

In [36]:
y_train_.shape

(9015,)

### Performance of baseline model

In [48]:
n_docs_class_ = np.array([y_train_[y_train_==i].shape[0] for i in range(1, 21)])
prior_ = n_docs_class_/np.sum(n_docs_class_)
likelihood_ = multinomials(X_train_, y_train_)

In [59]:
X_pred_valid_base = estimator(X_valid, prior_, likelihood_)

In [61]:
acc_base = X_pred_valid_base[X_pred_valid_base==y_valid].shape[0]/y_valid.shape[0]
print(f"Predicting on {y_valid.shape[0]} samples, Accuracy:{round(acc_base, 3)}")

Predicting on 2254 samples, Accuracy:0.84


Note: we don't remove stopwords here simply because the vocabulary has already eliminated common stopwords.

### strategy 1: replacing the frequency f of a word in a document by log(1+f)

$$tf(t,d) = log (1 + f_{t,d})$$

If a word appears again, the probability of it appearing again goes up. In order to smooth this, we take the log of the frequency:

In [66]:
def multinomials_revised(X_train, y_train):
    tf = np.zeros((n_classes, n_words))
    #X_train = X_train + 1
    for j in range(20):
        jth_class = X_train[y_train==j+1]
        f = (np.sum(jth_class, axis=0)+1)/(np.sum(jth_class) + n_words)
        tf[j] = np.log(1+f)
    return tf

In [67]:
log_likelihood = multinomials_revised(X_train_, y_train_)

In [68]:
X_pred_valid_1 = estimator(X_valid, prior_, log_likelihood)

In [69]:
acc_1 = X_pred_valid_1[X_pred_valid_1==y_valid].shape[0]/y_valid.shape[0]
print(f"Predicting on {y_valid.shape[0]} samples, Accuracy:{round(acc_1, 3)}")

Predicting on 2254 samples, Accuracy:0.84


### strategy 2: introducing ``Inverse Document Frequency``(IDF)

TF-IDF is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. It leverage the importance of some low-frequency words compared to high-frequency words with little relavence.

$$idf(t, D) = log \frac{N}{1+ |\{d\in D: t\in d\}|}$$

N: total number of documents in the corpus $N=|D|$

$|\{d\in D: t\in d\}|$: number of documents where the term t appears. If the term is not in the corpus, this will lead to a division by zero. Therefore it is common to adjust the denominator to $1+|\{d\in D: t\in d\}|$

$$Tfidf = tf(t, D)*idf(t, D)$$

In [45]:
#number of documents
N = np.array([X_train_[y_train_==i+1].shape[0] for i in range(20)])
N

array([376, 471, 447, 465, 462, 461, 468, 467, 490, 479, 486, 480, 471,
       480, 484, 473, 432, 447, 377, 299])

In [378]:
#np.count_nonzero(D, axis=0)

In [46]:
#calculate idf 
def inverse_term_frequency(X, y, n_words):
    n_classes = np.unique(y).shape[0]

    idf = np.zeros((n_classes, n_words))
    for i in range(20):
        D = X[y==i+1]
        N = np.array(D.shape[0])
        df = np.count_nonzero(D, axis=0) + 1 #add 1 for smoothing
        idf[i] = np.log(N/df)
        
    return idf

In [51]:
idf = inverse_term_frequency(X_train_, y_train_, n_words)

In [52]:
tf = multinomials(X_train_, y_train_)

In [53]:
tfidf = tf*idf

In [54]:
tfidf

array([[2.12860513e-04, 6.30249245e-04, 2.35394742e-03, ...,
        3.47263232e-05, 3.47263232e-05, 3.47263232e-05],
       [1.44526387e-03, 9.72199866e-04, 3.97351666e-05, ...,
        3.97351666e-05, 3.97351666e-05, 3.97351666e-05],
       [3.21619091e-04, 9.96811332e-04, 4.52111706e-05, ...,
        4.52111706e-05, 4.52111706e-05, 4.52111706e-05],
       ...,
       [1.44818061e-04, 8.78946355e-04, 2.36237524e-05, ...,
        2.36237524e-05, 2.36237524e-05, 2.36237524e-05],
       [2.92815901e-05, 4.00338502e-04, 2.92815901e-05, ...,
        2.92815901e-05, 2.92815901e-05, 2.92815901e-05],
       [3.68731634e-05, 4.63853338e-04, 1.94288775e-04, ...,
        3.68731634e-05, 3.68731634e-05, 3.68731634e-05]])

In [55]:
# calculate optimal solution for estimator
def estimator_tfidf(X_test, prior, tfidf):
    if X_test.ndim == 1:
        n_documents = 1
    else:
        n_documents = X_test.shape[0]

    labels = np.zeros((n_documents, ))
    for i in range(n_documents):
        log_prob = np.sum(np.log(tfidf)*X_test[i], axis=1) + np.log(prior)
        labels[i] = np.argmax(log_prob)+1
    return labels

In [56]:
X_pred_2 = estimator_tfidf(X_valid, prior_, tfidf)

In [57]:
X_pred_2.shape

(2254,)

In [58]:
acc_2 = X_pred_2[X_pred_2==y_valid].shape[0]/y_valid.shape[0]
print(f"Predicting on {y_valid.shape[0]} samples, Accuracy:{round(acc_2, 3)}")

Predicting on 2254 samples, Accuracy:0.761


Looks like there's no improvement by adding idf, this perhaps because the vocabulary set being dealing with has already eliminated all the high-frequency words that has little relavence to the documents.

<br/><br/>

Now we wraps everything up into class implementation.

# Class implementation of Naive-Bayes Multinomial Classifier

In [71]:
class NaiveBayes_clf():
    
    def __init__(self, alpha=1, tf='f', tfidf=False):
        self.alpha = alpha
        self.tf = tf
        self.tfidf = tfidf
        print(f"Naive-Bayes-Multinomial-Classifier: alpha={self.alpha}, tf={self.tf}, tfidf={self.tfidf}")
        
    def fit(self, X, y):
        self.n_classes = np.unique(y).shape[0]
        self.n_docs = X.shape[0]
        self.n_words = X.shape[1]
        
        #vector(20,), number of documents in each class
        n_docs_class = np.array([y[y==j].shape[0] for j in range(1, self.n_classes+1)]) #class starts from 1
        self.prior = n_docs_class/np.sum(n_docs_class)
        
        self.likelihood = self._multinomials(X, y)
        
    def _multinomials(self, X, y):
        tf = np.zeros((n_classes, n_words))

        for j in range(20):
            jth_class = X[y==j+1]
            f = (np.sum(jth_class, axis=0) + self.alpha)/(np.sum(jth_class) + self.alpha*n_words)
            if self.tf == 'f':
                tf[j] = f
            else:
                tf[j] = np.log(1+f)
        return tf
    

    # calculate optimal solution for estimator
    def predict(self, X):
        if X_test.ndim == 1:
            n_test_samples = 1
        else:
            n_test_samples = X_test.shape[0]

        labels = np.zeros((n_test_samples, ))
        for i in range(n_test_samples):
            log_prob = np.sum(np.log(self.likelihood)*X[i], axis=1) + np.log(self.prior)
            labels[i] = np.argmax(log_prob)+1
        return labels
    
    def accuracy(self, X, y):
        acc = X[X==y].shape[0]/y.shape[0]
        print(f"Predicting on {y.shape[0]} samples, Accuracy:{round(acc, 3)}")
        return acc

# Final testing

#### Predictions on test set with baseline model

In [74]:
clf_base = NaiveBayes_clf()
clf_base.fit(X_train, y_train)
X_pred_base = clf_base.predict(X_test)
clf_base.accuracy(X_pred_base, y_test)

Naive-Bayes-Multinomial-Classifier: alpha=1, tf=f, tfidf=False
Predicting on 7505 samples, Accuracy:0.781


0.7810792804796802

#### Predictions on test set with replaced log(1+f)

In [75]:
clf_log = NaiveBayes_clf(tf='log')
clf_log.fit(X_train, y_train)
X_pred_log = clf_log.predict(X_test)
clf_log.accuracy(X_pred_log, y_test)

Naive-Bayes-Multinomial-Classifier: alpha=1, tf=log, tfidf=False
Predicting on 7505 samples, Accuracy:0.781


0.7814790139906729